## 生成科研数据variant annotation的关系数据

In [3]:
import pandas as pd
import os
import json
import re
os.getcwd()

'D:\\pgkb_graph'

In [4]:
df_variant_annotation = pd.read_csv('variant_annotation/var_drug_ann.tsv', sep='\t', error_bad_lines=False).fillna("")
df_variant_annotation = df_variant_annotation[df_variant_annotation["Significance"] == "yes"]
df_variant_annotation.index = range(len(df_variant_annotation))
print(df_variant_annotation.shape)
df_variant_annotation[:2]

(5072, 11)


b'Skipping line 4308: expected 11 fields, saw 13\n'


,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population
0,1183684657,CYP2D6 ultrarapid metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 ultra-metabolizer genotype is associate...,,
1,1448997750,"CYP2B6*1, CYP2B6*18",CYP2B6,efavirenz,16495778,Metabolism/PK,yes,Please note that in the paper the allele was r...,CYP2B6 *1/*18 is associated with increased con...,*1/*18,


In [10]:
df_variant_param = pd.read_csv('variant_annotation/study_parameters.tsv', sep='\t', error_bad_lines=False).fillna("")
df_variant_param = df_variant_param[["Variant Annotation ID", "Study Type", "Study Cases", "Study Controls", "Characteristics",
                                     "Characteristics Type", "Frequency In Cases", "Frequency In Controls", "P Value", "Biogeographical Groups"]]
df_variant_merge = pd.merge(df_variant_annotation, df_variant_param, how="left", left_on="Variant Annotation ID", right_on="Variant Annotation ID")

# set 0.01 as p_value threshold to filter
p_value_list = df_variant_merge["P Value"].values
p_value_list = list(map(lambda x: "1" if ">" in str(x) else str(x).replace("<", "").replace("=", "").strip(), p_value_list))
p_value_bool_list = []
for x in p_value_list:
    try:
        if float(x) <= 0.01:
            p_value_bool_list.append(True)
        else:
            p_value_bool_list.append(False)
    except:
        p_value_bool_list.append(False)
df_variant_merge = df_variant_merge.assign(p_value_effect=p_value_bool_list)
df_variant_merge = df_variant_merge[df_variant_merge["p_value_effect"] == True]

print(df_variant_merge.shape)
df_variant_merge[:10]

(3344, 21)


,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,...,Study Type,Study Cases,Study Controls,Characteristics,Characteristics Type,Frequency In Cases,Frequency In Controls,P Value,Biogeographical Groups,p_value_effect
0,1183684657,CYP2D6 ultrarapid metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 ultra-metabolizer genotype is associate...,,...,cohort,14,,11 UM; 3 PM; healthy male non-smoking volunteers,Study Cohort,,,< 0.001,European,True
1,1448997750,"CYP2B6*1, CYP2B6*18",CYP2B6,efavirenz,16495778,Metabolism/PK,yes,Please note that in the paper the allele was r...,CYP2B6 *1/*18 is associated with increased con...,*1/*18,...,cohort,51,,,Unknown,,,< 0.0001,Multiple groups,True
2,1183684661,CYP2D6 normal metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 normal metabolizer genotype is associat...,,...,cohort,14,,11 UM; 3 PM; healthy male non-smoking volunteers,Study Cohort,,,< 0.001,European,True
3,1449191920,rs113993960,CFTR,ivacaftor / lumacaftor,28325531,Efficacy,yes,Study found a significant improvement in FEV i...,Genotype del/del is associated with response t...,del/del,...,cohort,53,,,Unknown,,,= 0.009,Unknown,True
4,1183632366,"CYP2D6*1, CYP2D6*4",CYP2D6,codeine,19940985,Metabolism/PK,yes,Lower plasma concentrations of the morphine me...,CYP2D6 *4/*4 is associated with decreased meta...,*4/*4,...,cohort,11,,End-stage renal disease patients with hemodial...,Study Cohort,,,= 0.01,Unknown,True
13,1449716526,"CYP3A4*1, CYP3A4*1G",CYP3A4,fentanyl,21223952,Metabolism/PK,yes,rs2242480 (*1G) was the only variant genotyped...,CYP3A4 *1G/*1G is associated with increased co...,*1G/*1G,...,cohort,176,,All participants were given fentanyl for analg...,Study Cohort,0.227,,< 0.01,East Asian,True
20,1451161540,"CYP2D6*1, CYP2D6*2, CYP2D6*2xN",CYP2D6,methadone,21589866,Metabolism/PK,yes,CYP2D6 ultrarapid metabolizers had significant...,CYP2D6 *1/*2xN + *2/*2xN (assigned as ultrarap...,*1/*2xN + *2/*2xN,...,cohort,105,,"P-value given for (R,S)-methadone concentrations",Study Cohort,,,= 0.002,European,True
21,1448994156,"CYP2C9*1, CYP2C9*3",CYP2C9,phenytoin,14659971,Metabolism/PK,yes,No *3/*3 individuals were observed.,CYP2C9 *1/*3 is associated with increased conc...,*1/*3,...,,28,,serum phenytoin concentration to drug dose,Drug,0.107,,= 0.01,East Asian,True
31,1451163180,rs4149056,SLCO1B1,atorvastatin,32128760,Metabolism/PK,yes,,Allele C is associated with increased exposure...,C,...,GWAS,590,,,Unknown,,,= 2.21e-6,European,True
32,1451163194,rs887829,"UGT1A1,""UGT1A10"",""UGT1A3"",""UGT1A4"",""UGT1A5"",""U...",atorvastatin,32128760,Metabolism/PK,yes,This variant was associated with increased met...,Allele T is associated with increased metaboli...,T,...,GWAS,590,,,Unknown,,,= 7.25e-16,European,True


In [6]:
df_variant_merge.columns

Index(['Variant Annotation ID', 'Variant/Haplotypes', 'Gene', 'Drug(s)',
       'PMID', 'Phenotype Category', 'Significance', 'Notes', 'Sentence',
       'Alleles', 'Specialty Population', 'Study Type', 'Study Cases',
       'Study Controls', 'Characteristics', 'Characteristics Type',
       'Frequency In Cases', 'Frequency In Controls', 'P Value',
       'Biogeographical Groups', 'p_value_effect'],
      dtype='object')

In [7]:
# list(filter(lambda x: "," in x, df_variant_merge["Variant/Haplotypes"].values))

In [46]:
variant_drug_research_list = []
diplotype_drug_research_list = []

for index, row in df_variant_merge.iterrows():
    variant = row["Variant/Haplotypes"]
    if "genotype" in variant:
        continue
    variant_list = filter(lambda x: x!= "", [x.strip() for x in variant.split(",")])
        
    drugs = row["Drug(s)"]
    drug_list = list(filter(lambda x: x!= "", [x.replace("\"", "").strip() for x in re.split("/|,", drugs)]))
    p_value = "P value {}".format(row["P Value"]).replace("=", "").replace("<", "")
    phenotype = row["Phenotype Category"]
    phenotype_list = set([x.replace("\"", "").strip() for x in phenotype.split(",")])
    PMID = row["PMID"]
    note = row["Notes"]
    sentence = row["Sentence"]
    biogeo_group = row["Biogeographical Groups"].replace("\"", "'")            
    
    if row["Alleles"] != "" and "/" in row["Alleles"]:
        diplotype_list = [x.strip() for x in row["Alleles"].split("+")]
        for d in drug_list:
            for dip in diplotype_list:
                for p in phenotype_list:
                    if "/" in dip:
                        diplotype = "{} {}".format(row["Gene"], dip)
                        diplotype_drug_research_list.append((d, p, p_value, biogeo_group, PMID, note, sentence, diplotype))
    else:
        for d in drug_list:
            for v in variant_list:
                for p in phenotype_list:
                    variant_drug_research_list.append((d, p, p_value, biogeo_group, PMID, note, sentence, v))

In [47]:
print(len(variant_drug_research_list))
variant_drug_research_list[:5]

3147


[('atorvastatin',
  'Metabolism/PK',
  'P value  2.21e-6',
  'European',
  32128760,
  '',
  'Allele C is associated with increased exposure to atorvastatin as compared to allele T.',
  'rs4149056'),
 ('atorvastatin',
  'Metabolism/PK',
  'P value  7.25e-16',
  'European',
  32128760,
  'This variant was associated with increased metabolic ratios of both 2-OH ATV/ATV (P = 7.25 × 10−16) and 2-OH ATV L/ATV L (P = 3.95 × 10−15) at genome-wide significance. This variant is associated with increased hydroxylation of ATV, and not lactonization.',
  'Allele T is associated with increased metabolism of atorvastatin as compared to allele C.',
  'rs887829'),
 ('atorvastatin',
  'Metabolism/PK',
  'P value  6.18e-7',
  'European',
  32128760,
  'This variant was associated with increased metabolic ratios of 2-OH ATV/ATV (P = 6.18 × 10−7) (increased hydroxylation of ATV) at genome-wide significance.',
  'Allele G is associated with increased metabolism of atorvastatin as compared to allele T.',
  

In [48]:
print(len(diplotype_drug_research_list))
diplotype_drug_research_list[:5]

1394


[('efavirenz',
  'Metabolism/PK',
  'P value  0.0001',
  'Multiple groups',
  16495778,
  'Please note that in the paper the allele was referred to as CYP2B6*16. CYP2B6*16 and *18 alleles have been consolidated by PharmVar in Jan 2020, with *16 now listed as a suballele of *18 (CYP2B6*18.002). This annotation is updated to be on CYP2B6*18, instead of CYP2B6*16.',
  'CYP2B6 *1/*18 is associated with increased concentrations of efavirenz in people with HIV Infections as compared to CYP2B6 *1/*1.',
  'CYP2B6 *1/*18'),
 ('ivacaftor',
  'Efficacy',
  'P value  0.009',
  'Unknown',
  28325531,
  "Study found a significant improvement in FEV in all patients following 3 months of treatment. There was no difference in patients' BMI.",
  'Genotype del/del is associated with response to ivacaftor / lumacaftor in people with Cystic Fibrosis.',
  'CFTR del/del'),
 ('lumacaftor',
  'Efficacy',
  'P value  0.009',
  'Unknown',
  28325531,
  "Study found a significant improvement in FEV in all patient

In [49]:
pd.DataFrame(variant_drug_research_list, 
             columns=["drug", "phenotype_category", "p_value", "bio_geo_group", "PMID", "note", "sentence", "variant"]
            ).assign(data_source=["research"] * len(variant_drug_research_list)
                    ).to_csv("processed/research_drug_variant_annotation.csv", index=False)

pd.DataFrame(diplotype_drug_research_list, 
             columns=["drug", "phenotype_category", "p_value", "bio_geo_group", "PMID", "note", "sentence", "diplotype"]
            ).assign(data_source=["research"] * len(diplotype_drug_research_list)
                    ).to_csv("processed/research_drug_diplotype_annotation.csv", index=False)